In [ ]:
# SPDX-FileCopyrightText: Copyright (c) 2022 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
# SPDX-License-Identifier: MIT

# NLP API tutorial

This tutorial demonstates how to use Python Riva API.

## <font color="blue">Server</font>

Before running client part of Riva, please set up a server. The simplest
way to do this is to follow
[quick start guide](https://docs.nvidia.com/deeplearning/riva/user-guide/docs/quick-start-guide.html#local-deployment-using-quick-start-scripts).


## <font color="blue">Authentication</font>

Before using Riva services you will need to establish connection with a server.

In [ ]:
import riva.client

uri = "localhost:50051"  # Default value

auth = riva.client.Auth(uri=uri)

## <font color="blue">Setting up service</font>

To instantiate a service pass `riva.client.Auth` instance to a constructor.

In [ ]:
nlp_service = riva.client.NLPService(auth)

You may find full response field description in [documentation](https://docs.nvidia.com/deeplearning/riva/user-guide/docs/reference/protos/riva_asr.proto.html?highlight=max%20alternatives#riva-proto-riva-asr-proto).

In [ ]:
from riva.client.proto.riva_nlp_pb2 import (
    AnalyzeIntentResponse,
    NaturalQueryResponse,
    TextClassResponse,
    TextTransformResponse,
    TokenClassResponse,
)

## <font color="blue">Text classification</font>

In [ ]:
text_class_queries = ["A hurricane is approaching Japan.", "What is the weather on Wednesday in Moscow?"]
text_class_model = "riva_intent_weather"

In [ ]:
response: TextClassResponse = nlp_service.classify_text(text_class_queries, text_class_model)

In [ ]:
print(response)

In [ ]:
detected_intent = response.results[0].labels[0].class_name

In [ ]:
print(detected_intent)

You may use a function `riva.client.extract_most_probable_text_class_and_confidence()`. The function returns a list of most probable intents and their scores for all queries.

In [ ]:
classes, probs = riva.client.extract_most_probable_text_class_and_confidence(response)

In [ ]:
print(classes)
print(probs)

## <font color="blue">Token classification</font>

In [ ]:
response: TokenClassResponse = nlp_service.classify_tokens(text_class_queries[1], text_class_model)

In [ ]:
print(response)

In [ ]:
token = response.results[0].results[0].token
class_name = response.results[0].results[0].label[0].class_name
class_score = response.results[0].results[0].label[0].score

In [ ]:
print(token, class_name, class_score)

You may use a function `riva.client.extract_most_probable_token_classification_predictions()`. The function returns:
 - list of tokens lists for all elements of a batch,
 - list of most probable classes lists for all elements of a batch,
 - list of most probable classes confidences lits for all elements of a batch,
 - list of token span starts lists for all elements of a batch,
 - list of token span ends lists for all elements of a batch.

In [ ]:
tokens, class_names, confidences, starts, ends = riva.client.extract_most_probable_token_classification_predictions(
    response
)

In [ ]:
print("First batch element tokens:", tokens[0])
print("First batch element first token class name:", class_names[0][0])
print(confidences)
print(starts)
print(ends)

> Spans do not work properly for batches which contain more than 1 element.

## <font color="blue">Punctuation and Capitalization</font>

In [ ]:
# Batches of sizes greater than 1 are not working with riva v2.2.0 and v2.2.1

punctuation_queries = [
    "by the early 20th century the gar complained more and more about the younger generation",
#     "boa Vista is the capital of the brazilian state of roraima situated on the western bank of "
#     "the branco river the city lies 220 km from brazil's border with venezuela.",
]
response: TextTransformResponse = nlp_service.punctuate_text(punctuation_queries)

In [ ]:
print(response)

In [ ]:
first_query_result = response.text[0]
print(first_query_result)

## <font color="blue">Intent analysis</font>

Accepts an input string and returns the most likely intent as well as slots relevant to that intent.

In [ ]:
options = riva.client.AnalyzeIntentOptions(lang='en-US')
intent_query = "How is the weather today in New England?"
response: AnalyzeIntentResponse = nlp_service.analyze_intent(intent_query, options)

In [ ]:
print(response)

In [ ]:
print("intent name:", response.intent.class_name)
print("intent score:", response.intent.score)
print("domain name:", response.domain.class_name)
print("domain score:", response.domain.score)
print("first slot token:", response.slots[0].token)
print("first slot most probable label name:", response.slots[0].label[0].class_name)
print("first slot most probable label score:", response.slots[0].label[0].score)

## <font color="blue">Question answering</font>

In [ ]:
qa_query = "How many gigatons of carbon dioxide was released in 2005?"
qa_context = (
    "In 2010 the Amazon rainforest experienced another severe drought, in some ways more extreme than the "
    "2005 drought. The affected region was approximate 1,160,000 square miles (3,000,000 km2) of "
    "rainforest, compared to 734,000 square miles (1,900,000 km2) in 2005. The 2010 drought had three "
    "epicenters where vegetation died off, whereas in 2005 the drought was focused on the southwestern "
    "part. The findings were published in the journal Science. In a typical year the Amazon absorbs 1.5 "
    "gigatons of carbon dioxide; during 2005 instead 5 gigatons were released and in 2010 8 gigatons were "
    "released."
)
response: NaturalQueryResponse = nlp_service.natural_query(qa_query, qa_context)

In [ ]:
print(response)

In [ ]:
answer = response.results[0].answer
score = response.results[0].score

In [ ]:
print(answer)

In [ ]:
print(score)

## <font color="blue">Asynchronous calls</font>

Any of the above methods can be used in asynchronous manner. For this you need set parameter `future=True`. Then instead of response the methods will return future objects. Responses can be retrieved by calling `result()` on future objects.

Following example demonstrates how latency is reduced via using asynchronous calls.

In [ ]:
from time import time

In [ ]:
text_class_queries = ["A hurricane is approaching Japan.", "What is the weather on Wednesday in Moscow?"]
text_class_model = "riva_intent_weather"
# Batches of sizes greater than 1 are not working with riva v2.2.0 and v2.2.1

punctuation_queries = [
    "by the early 20th century the gar complained more and more about the younger generation",
#     "boa Vista is the capital of the brazilian state of roraima situated on the western bank of "
#     "the branco river the city lies 220 km from brazil's border with venezuela.",
]

options = riva.client.AnalyzeIntentOptions(lang='en-US')
intent_query = "How is the weather today in New England?"

qa_query = "How many gigatons of carbon dioxide was released in 2005?"
qa_context = (
    "In 2010 the Amazon rainforest experienced another severe drought, in some ways more extreme than the "
    "2005 drought. The affected region was approximate 1,160,000 square miles (3,000,000 km2) of "
    "rainforest, compared to 734,000 square miles (1,900,000 km2) in 2005. The 2010 drought had three "
    "epicenters where vegetation died off, whereas in 2005 the drought was focused on the southwestern "
    "part. The findings were published in the journal Science. In a typical year the Amazon absorbs 1.5 "
    "gigatons of carbon dioxide; during 2005 instead 5 gigatons were released and in 2010 8 gigatons were "
    "released."
)

In [ ]:
text_class_reference = ['weather.alert', 'weather.weather']
token_class_reference = ['wednesday', 'moscow', '?']
punctuation_reference = (
    "By the early 20th century, the Gar complained more and more about the younger generation."
)
intent_analysis_reference = "weather.weather"
natural_query_reference = "5"

In [ ]:
num_repeats = 10

In [ ]:
from typing import List

def check_text_classification(ref: List[str], resp: TextClassResponse, repeat_idx: int) -> None:
    labels, _ = riva.client.extract_most_probable_text_class_and_confidence(resp)
    assert labels == ref, f"On repeat {repeat_idx} expected text classification results {ref}, but got {labels}."

def check_token_classification(ref: List[str], resp: TokenClassResponse, repeat_idx: int) -> None:
    tokens = riva.client.extract_most_probable_token_classification_predictions(resp)[0][0]
    assert tokens == token_class_reference, (
        f"On repeat {repeat_idx} expected to find token classification tokens {ref}, but got {tokens}."
    )

def check_punctuation(ref: str, resp: TextTransformResponse, repeat_idx: int) -> None:
    output = resp.text[0]
    assert output == ref, (
        f"On repeat {repeat_idx} expected punctuated output '{ref}', but got '{output}'."
    )

def check_intent_analysis(ref: str, resp: AnalyzeIntentResponse, repeat_idx: int) -> None:
    output = resp.intent.class_name
    assert output == ref, f"On repeat {repeat_idx} expected intent is '{ref}', but got '{output}'."

def check_natural_query(ref: str, resp: NaturalQueryResponse, repeat_idx: int) -> None:
    answer = resp.results[0].answer
    assert answer == ref, f"On repeat {repeat_idx} expected answer is '{ref}', but got '{answer}'."

Synchronous:

In [ ]:
start_time = time()
for repeat_idx in range(num_repeats):
    text_class_response = nlp_service.classify_text(text_class_queries, text_class_model)
    check_text_classification(text_class_reference, text_class_response, repeat_idx)
    
    token_class_response = nlp_service.classify_tokens(text_class_queries[1], text_class_model)
    check_token_classification(token_class_reference, token_class_response, repeat_idx)
    
    punctuation_response = nlp_service.punctuate_text(punctuation_queries)
    check_punctuation(punctuation_reference, punctuation_response, repeat_idx)
    
    intent_analysis_response = nlp_service.analyze_intent(intent_query, options)
    check_intent_analysis(intent_analysis_reference, intent_analysis_response, repeat_idx)
    
    natural_query_response = nlp_service.natural_query(qa_query, qa_context)
    check_natural_query(natural_query_reference, natural_query_response, repeat_idx)
    
print(f"time spent on synchronous calls: {time() - start_time:.2f} sec")

Asynchronous:

In [ ]:
start_time = time()
futures = []
for _ in range(num_repeats):
    repeat_futures = []
    repeat_futures.append(nlp_service.classify_text(text_class_queries, text_class_model, future=True))
    repeat_futures.append(nlp_service.classify_tokens(text_class_queries[1], text_class_model, future=True))
    repeat_futures.append(nlp_service.punctuate_text(punctuation_queries, future=True))
    repeat_futures.append(nlp_service.analyze_intent(intent_query, options, future=True))
    repeat_futures.append(nlp_service.natural_query(qa_query, qa_context, future=True))
    futures.append(repeat_futures)
for repeat_idx, repeat_futures in enumerate(futures):
    check_text_classification(text_class_reference, repeat_futures[0].result(), repeat_idx)
    check_token_classification(token_class_reference, repeat_futures[1].result(), repeat_idx)
    check_punctuation(punctuation_reference, repeat_futures[2].result(), repeat_idx)
    check_intent_analysis(intent_analysis_reference, repeat_futures[3].result(), repeat_idx)
    check_natural_query(natural_query_reference, repeat_futures[4].result(), repeat_idx)
        
print(f"time spent on async calls: {time() - start_time:.2f}sec")